In [3]:
!pip install gradio --quiet

import numpy as np
import pandas as pd
import gradio as gr
from sklearn.linear_model import LinearRegression

# --- Athlete Dataset ---
data = [
    ['M', 17, 75, 170, 185, 200, 28, 13, 2, 6, 4.5, 11.9, 24.0, 52.4],
    ['M', 17, 75, 170, 155, 275, 27, 12, 1, 6, 4.3, 12.3, 24.5, 54.0],
    ['M', 27, 71, 154, 225, 315, 36, 8, 5, 9, 3.8, 9.79, 19.31, 45.87],
    ['M', 18, 71, 145, 185, 315, 30, 5, 5, 7, 4.4, 11.4, 23.3, 52.4],
    ['M', 19, 65, 120, 115, 185, 22, 16, 0, 4, 5.5, 13.5, 27.5, 61.0],
    ['M', 18, 79, 215, 275, 405, 34, 10, 5, 9, 3.5, 10.5, 21.5, 48.0],
    ['M', 17, 72, 160, 135, 225, 26, 14, 1, 3, 5.2, 12.9, 26.1, 58.0],
    ['M', 16, 68, 140, 95, 185, 24, 18, 0, 2, 5.8, 14.0, 28.0, 62.0],
    ['M', 18, 74, 180, 205, 315, 32, 9, 6, 9, 3.6, 10.9, 21.2, 47.9],
    ['M', 18, 69, 150, 185, 265, 29, 12, 3, 6, 4.6, 12.0, 25.1, 55.0]
]

df = pd.DataFrame(data, columns=[
    'Gender','Age','Height_in','Weight_lbs','Bench_lbs','Squat_lbs','Vertical_in',
    'BodyFat','SprintDays','Stamina','TopSpeedTime','100m','200m','400m'
])

# Feature engineering
df['Gender'] = 1
df['Squat_to_weight'] = df['Squat_lbs'] / df['Weight_lbs']
df['Explosive_index'] = df['Vertical_in'] + df['Bench_lbs'] / df['Weight_lbs']
df['ShortPenalty'] = df['Height_in'].apply(lambda x: max(0, 70 - x) * 0.1)
df['SprintDeficit'] = df['SprintDays'].apply(lambda x: max(0, 3 - x) * 0.5)

X = df[['Gender','Age','Height_in','Weight_lbs','Squat_lbs','Bench_lbs',
        'Vertical_in','BodyFat','SprintDays','Stamina','TopSpeedTime',
        'Squat_to_weight','Explosive_index','ShortPenalty','SprintDeficit']]

y_100 = df['100m']
y_200 = df['200m']
y_400 = df['400m']

model_100 = LinearRegression().fit(X, y_100)
model_200 = LinearRegression().fit(X, y_200)
model_400 = LinearRegression().fit(X, y_400)

# --- Prediction function ---
def predict(age, feet, inches, weight, bench, squat, vertical, bodyfat, sprint_days, stamina, topspeed_time):
    g = 1
    height_in = feet * 12 + inches
    squat_to_weight = squat / weight
    explosive_index = vertical + bench / weight
    short_penalty = max(0, 70 - height_in) * 0.1
    sprint_deficit = max(0, 3 - sprint_days) * 0.5

    features = np.array([[g, age, height_in, weight, squat, bench, vertical,
                          bodyfat, sprint_days, stamina, topspeed_time,
                          squat_to_weight, explosive_index, short_penalty, sprint_deficit]])

    pred_100 = model_100.predict(features)[0] + 0.4      # tuned
    pred_200 = model_200.predict(features)[0] - 0.4      # fixed to be faster
    pred_400 = model_400.predict(features)[0]            # left as is

    return (
        f"{pred_100:.2f} sec",
        f"{pred_200:.2f} sec",
        f"{pred_400:.2f} sec"
    )

# --- Gradio Interface ---
with gr.Blocks(css="body { font-family: 'Segoe UI', sans-serif; background-color: #f9fbfd; }") as demo:
    gr.Markdown(
        """
        # 🏃‍♂️ **TrackAI: Sprint Time Predictor for Male Athletes**
        _Made by Ranvir Singh_

        Predict your 100m, 200m, and 400m times based on body stats, strength, and training.
        """
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 🧍 Body & Strength")
            age = gr.Slider(14, 30, step=1, label="Age")
            feet = gr.Slider(4, 7, step=1, label="Height (feet)")
            inches = gr.Slider(0, 11, step=1, label="Height (inches)")
            weight = gr.Slider(100, 250, step=1, label="Weight (lbs)")
            bodyfat = gr.Slider(3, 30, step=1, label="Body Fat (%)")
            bench = gr.Slider(50, 350, step=5, label="Bench Max (lbs)")
            squat = gr.Slider(80, 500, step=5, label="Squat Max (lbs)")

        with gr.Column():
            gr.Markdown("### 🏃 Speed & Conditioning")
            vertical = gr.Slider(15, 40, step=1, label="Vertical Jump (inches)")
            sprint_days = gr.Slider(0, 7, step=1, label="Sprint Days per Week")
            stamina = gr.Slider(1, 10, step=1, label="Stamina (1–10)")
            topspeed_time = gr.Slider(2.5, 6.0, step=0.1, label="Time to Reach Top Speed (sec)")

    predict_btn = gr.Button("🚀 Predict Times")
    out_100 = gr.Text(label="Predicted 100m Time")
    out_200 = gr.Text(label="Predicted 200m Time")
    out_400 = gr.Text(label="Predicted 400m Time")

    predict_btn.click(fn=predict, inputs=[
        age, feet, inches, weight, bench, squat, vertical, bodyfat,
        sprint_days, stamina, topspeed_time
    ], outputs=[out_100, out_200, out_400])

demo.launch()









It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ca2650748473237763.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
